In [2]:
import pandas as pd

df = pd.read_csv('flightdata.csv')
df.head()


df.shape
df.isnull().values.any()

df.isnull().sum()

df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]
df.isnull().sum()

df[df.isnull().values.any(axis=1)].head()

df = df.fillna({'ARR_DEL15': 1})

df.iloc[177:185]
df.head()

import math

for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100)
df.head()

df = pd.get_dummies(df, columns=['ORIGIN', 'DEST'])
df.head()

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop('ARR_DEL15', axis=1), df['ARR_DEL15'], test_size=0.2, random_state=42)

train_x.shape
test_x.shape

import pickle
look_back=1
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
import time #helper libraries

model = Sequential()
model.add(LSTM(25, input_shape=(14, look_back)))
model.add(Dropout(0.1))
model.add(Dense(20000))
model.compile(loss='mse', optimizer='adam')
model.fit(train_x, train_y, epochs=10, batch_size=240, verbose=1)

loss_and_metrics = model.evaluate(test_x,test_y)
k=(10-float(loss_and_metrics))*10
print("Accuracy : ",k)
filename = 'LSTM.sav'
pickle.dump(model, open(filename, 'wb'))
y_pred = model.predict(test_x)

predicted = model.predict(test_x)
print(model.evaluate(test_x, test_y))


def predict_delay(departure_date_time, origin, destination):
    from datetime import datetime

    try:
        departure_date_time_parsed = datetime.strptime(departure_date_time, '%d/%m/%Y %H:%M:%S')
    except ValueError as e:
        return 'Error parsing date/time - {}'.format(e)

    month = departure_date_time_parsed.month
    day = departure_date_time_parsed.day
    day_of_week = departure_date_time_parsed.isoweekday()
    hour = departure_date_time_parsed.hour

    origin = origin.upper()
    destination = destination.upper()

    input = [{'MONTH': month,
              'DAY': day,
              'DAY_OF_WEEK': day_of_week,
              'CRS_DEP_TIME': hour,
              'ORIGIN_ATL': 1 if origin == 'ATL' else 0,
              'ORIGIN_DTW': 1 if origin == 'DTW' else 0,
              'ORIGIN_JFK': 1 if origin == 'JFK' else 0,
              'ORIGIN_MSP': 1 if origin == 'MSP' else 0,
              'ORIGIN_SEA': 1 if origin == 'SEA' else 0,
              'DEST_ATL': 1 if destination == 'ATL' else 0,
              'DEST_DTW': 1 if destination == 'DTW' else 0,
              'DEST_JFK': 1 if destination == 'JFK' else 0,
              'DEST_MSP': 1 if destination == 'MSP' else 0,
              'DEST_SEA': 1 if destination == 'SEA' else 0 }]
    # printing the resultant list of a dictionary keys
    rt=model.predict(pd.DataFrame(input))[0][0]
    return rt





Epoch 1/10
38/38 [==============================] - 5s 65ms/step - loss: 0.1277
Epoch 2/10
38/38 [==============================] - 2s 62ms/step - loss: 0.1201
Epoch 3/10
38/38 [==============================] - 2s 61ms/step - loss: 0.1202
Epoch 4/10
38/38 [==============================] - 4s 94ms/step - loss: 0.1201
Epoch 5/10
38/38 [==============================] - 4s 107ms/step - loss: 0.1203
Epoch 6/10
38/38 [==============================] - 3s 63ms/step - loss: 0.1201
Epoch 7/10
38/38 [==============================] - 2s 62ms/step - loss: 0.1198
Epoch 8/10
38/38 [==============================] - 2s 63ms/step - loss: 0.1200
Epoch 9/10
38/38 [==============================] - 2s 62ms/step - loss: 0.1200
Epoch 10/10
71/71 [==============================] - 1s 6ms/step - loss: 0.1191
Accuracy :  98.808863312006


71/71 [==============================] - 1s 9ms/step - loss: 0.1191
0.11911366879940033


In [2]:
a=input("Enter date and Time in this Format 1/10/2018 3:45:00")
b=input("Enter orgin")
c=input("Enter dest")


xd=predict_delay(a,b, c)
#j="<html><head></head><body><h1><center>"+str(xd*100)+" MIN DELAY"+"</center></h1></body></html>"
j=str(xd*100)+"_MIN_DELAY"
print(j)
import urllib.request
yu="https://api.thingspeak.com/update?api_key=8JU3M80YH7V1UITT&field1="+str(j)
fg=urllib.request.urlopen(yu)

Enter date and Time in this Format 1/10/2018 3:45:001/10/2018 3:45:00
Enter orginSEA
Enter destMSP
1/1 [==============================] - 0s 22ms/step
9.094405174255371_MIN_DELAY
